In [27]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from datasets import load_from_disk
from pathlib import Path
import matplotlib.pyplot as plt

In [28]:
project_root = Path.cwd().parent
data_path = project_root / "data"  / "train"

dataset = load_from_disk(str(data_path))
sample = dataset[0]  
image = sample["image"] 
question = sample["question"]
answer = sample["answer"]

In [29]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 12.06it/s]


In [30]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")


In [31]:
min_pixels = 256*28*28
max_pixels = 1280*28*28

In [32]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": question}
        ]
    }
]

In [33]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [34]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [38]:
inputs = inputs.to("cuda")

In [39]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

AttributeError: 'dict' object has no attribute 'input_ids'

In [37]:
print(output_text)

cuda
